### PART 2: Connect to Spotify API

_______________________

In this notebook, we will explore the spotify API, extract songs from playlists, which will be used to build a database of music.

_____________________

#### Import Libraries

In [73]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import random
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA    
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
import numpy as np

from random import randint
from time import sleep

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#### Import Spotify API credentials.

In [80]:
secrets_file = open("secrets_p.txt","r")

In [81]:
string = secrets_file.read()
#string.split('\n')

In [82]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [83]:
#secrets_dict

#### Authentication with secrets

In [84]:
#InitializeSpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

#### Extract song information from spotify "spotipy" api wrapper

In [28]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [29]:
%%time
playlist1 = get_playlist_tracks("6tIxyT1Gq6O7DK7rIEUEZo")
playlist2 = get_playlist_tracks("4pbDDX7np7Q1H0ghL7U9o7")

CPU times: user 2.04 s, sys: 365 ms, total: 2.4 s
Wall time: 5min 33s


In [30]:
#playlist1

In [31]:
#total songs (probably we have to check for duplicates)

print('no. 1    ',len(playlist1))
print('no. 2    ',len(playlist2))
print('total is',len(playlist1)+len(playlist2))

no. 1     9999
no. 2     3878
total is 13877


#### Extract and transform the features, transforming everything into a dataframe

In [32]:
from tqdm import tqdm

def artist_song(x):
    artist = []
    song = []
    features = []
    counter = 0
    for i in tqdm(x):
        song.append(x[counter]['track']['name'])
        artist.append(x[counter]['track']['artists'][0]['name'])
        features.append(x[counter]["track"]["uri"])
        
        #features.append(sp.audio_features(x[counter]["track"]["uri"]))
        counter += 1
    return pd.DataFrame({'artist': artist, 'song': song, 'features': features})

#### Now we use the function we just created in both lists and export them to .csv files.

In [33]:
%%time
df_playlist2 = artist_song(playlist2)

100%|███████████████████████████████████| 3878/3878 [00:00<00:00, 194043.60it/s]

CPU times: user 26.4 ms, sys: 23.3 ms, total: 49.7 ms
Wall time: 52.4 ms


In [34]:
df_playlist2

,artist,song,features
0,ANGRA,Live and Learn,spotify:track:5LZci9wxIwwvF9qh8cMpHS
1,ANGRA,Bleeding Heart,spotify:track:4m2HvlrZ1kKsgFpcLh5h0V
2,ANGRA,Hunters and Prey,spotify:track:3Gs5F9IN5BEhU8jhdr5KrQ
3,ANGRA,Eyes of Christ,spotify:track:2ahYaznOR2pfAu6voMj3T7
4,ANGRA,Rebirth (Acoustic Version),spotify:track:3tQWuqbfcQumQL3KHfOM3C
...,...,...,...
3873,Fleetwood Mac,Family Man,spotify:track:7FaarqLNsZlFsRMQjjX5QR
3874,Fleetwood Mac,Welcome To The Room...Sara,spotify:track:5H1EXMKjBFlI5F9QkCD6Ft
3875,Fleetwood Mac,Isn't It Midnight,spotify:track:2icOTC7sCLZSDVbCPJ6olG
3876,Fleetwood Mac,When I See You Again,spotify:track:0QE9gbtUuiSJIqclW9N2TQ


In [35]:
df_playlist2.to_csv('playlist2.csv',index = False)

In [36]:
%%time
df_playlist1 = artist_song(playlist1)

100%|████████████████████████████████████| 9999/9999 [00:00<00:00, 91204.12it/s]

CPU times: user 54.5 ms, sys: 24.2 ms, total: 78.8 ms
Wall time: 128 ms


In [37]:
df_playlist1.to_csv('playlist1.csv',index = False)

In [38]:
df_playlist1

,artist,song,features
0,Queen,A Kind Of Magic - Remastered 2011,spotify:track:5RYLa5P4qweEAKq5U1gdcK
1,Queen,Bohemian Rhapsody - Remastered 2011,spotify:track:4u7EnebtmKWzUH433cf5Qv
2,Queen,Another One Bites The Dust - Remastered 2011,spotify:track:5vdp5UmvTsnMEMESIF2Ym7
3,Queen,Don't Stop Me Now - Remastered 2011,spotify:track:5T8EDUDqKcs6OSOwEsfqG7
4,Queen,Under Pressure - Remastered 2011,spotify:track:2fuCquhmrzHpu5xcA1ci9x
...,...,...,...
9994,Kip Winger,California - From the Moon to the Sun,spotify:track:2STE3vIHHIgz9TNB5aJGC1
9995,Kip Winger,What We Are - From the Moon to the Sun,spotify:track:3pQJKvK6hmVPzDXRv67PCo
9996,Kip Winger,One Big Game - From the Moon to the Sun,spotify:track:03P0IMmk5nXRyrhh0YpTfc
9997,Kip Winger,Why - From the Moon to the Sun,spotify:track:70Hf4cnE286JKpi3lwRb1w


#### Extracting features information

In [43]:
newlist = []
counter = 0
for i in tqdm(df_playlist_1_1['features']):
    newlist.append(sp.audio_features(i))
    
    if counter > 50:
        sleep(randint(10,25))
        counter = 0
    else:
        counter +=1

 67%|██████████████████████████▎            | 6737/9999 [45:54<22:13,  2.45it/s]


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=0sny9Sbdgd0gFxMZuND4QA:
 Max Retries, reason: too many 429 error responses

#### Split df into 2 parts


In [49]:
df_playlist_1_1 = df_playlist1.loc[0:6736]
df_playlist_1_2 = df_playlist1.loc[6737:]

In [53]:
df_playlist_1_1['features'] = newlist

/var/folders/75/lxv2pt816y3g7mh_b0zy07mw0000gn/T/ipykernel_1311/2435714390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_playlist_1_1['features'] = newlist


,artist,song,features
0,Queen,A Kind Of Magic - Remastered 2011,"[{'danceability': 0.67, 'energy': 0.776, 'key'..."
1,Queen,Bohemian Rhapsody - Remastered 2011,"[{'danceability': 0.414, 'energy': 0.404, 'key..."
2,Queen,Another One Bites The Dust - Remastered 2011,"[{'danceability': 0.933, 'energy': 0.528, 'key..."
3,Queen,Don't Stop Me Now - Remastered 2011,"[{'danceability': 0.559, 'energy': 0.868, 'key..."
4,Queen,Under Pressure - Remastered 2011,"[{'danceability': 0.671, 'energy': 0.712, 'key..."
...,...,...,...
6732,Sonata Arctica,Tallulah,"[{'danceability': 0.508, 'energy': 0.613, 'key..."
6733,Sonata Arctica,Paid In Full,"[{'danceability': 0.515, 'energy': 0.913, 'key..."
6734,Suld,Princess Xixia,"[{'danceability': 0.512, 'energy': 0.953, 'key..."
6735,Suld,The Memory of Nomadism,"[{'danceability': 0.524, 'energy': 0.942, 'key..."


In [55]:
DO NOT UN HASH THIS
#df_playlist_1_1.to_csv('df_playlist_1_1.csv', index = False)

In [74]:
df_playlist_1_2 = df_playlist_1_2.reset_index(drop = True)
df_playlist_1_2

,artist,song,features
0,Vanir,Raise Your Horns,spotify:track:0sny9Sbdgd0gFxMZuND4QA
1,Voloh,Izdajice,spotify:track:6OSDixPbDnTJZ7WnEC4fqH
2,Voloh,Posljednji pohod,spotify:track:1A5CU1K7coTsmLWQ3XxLGa
3,Circus Maximus,Glory of the Empire,spotify:track:1PrSlUPEtnwTGXlL2kQF6u
4,Dream Theater,The Silent Man,spotify:track:2vzfV6LgfAWtv7JeSoEPFO
...,...,...,...
3257,Kip Winger,California - From the Moon to the Sun,spotify:track:2STE3vIHHIgz9TNB5aJGC1
3258,Kip Winger,What We Are - From the Moon to the Sun,spotify:track:3pQJKvK6hmVPzDXRv67PCo
3259,Kip Winger,One Big Game - From the Moon to the Sun,spotify:track:03P0IMmk5nXRyrhh0YpTfc
3260,Kip Winger,Why - From the Moon to the Sun,spotify:track:70Hf4cnE286JKpi3lwRb1w


In [85]:

newlist2 = []
counter = 0
for i in tqdm(df_playlist_1_2['features']):
    newlist2.append(sp.audio_features(i))
    
    if counter > 50:
        sleep(randint(10,25))
        counter = 0
    else:
        counter +=1

  0%|                                                  | 0/3262 [00:02<?, ?it/s]


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=0sny9Sbdgd0gFxMZuND4QA:
 Max Retries, reason: too many 429 error responses

In [ ]:
len(newlist2)

In [ ]:
df_playlist_1_2['features'] = newlist2

In [ ]:
#USE ONCE, DO NOT UN HASH THIS AFTER
df_playlist_1_2.to_csv('df_playlist_1_2.csv', index = False)

In [ ]:

newlist3 = []
counter = 0
for i in tqdm(df_playlist_2['features']):
    newlist3.append(sp.audio_features(i))
    
    if counter > 50:
        sleep(randint(10,25))
        counter = 0
    else:
        counter +=1

In [ ]:
df_playlist_2['features'] = newlist3

In [ ]:
df_playlist_2.to_csv('df_playlist_2.csv', index = False)

#### Now we normalise using the json_normalize() function, then we apply them to individual columns and convert them to a dataframe.

In [ ]:
df_playlist1['features'] = json_normalize(df_playlist1['features'])
df_playlist2['features'] = json_normalize(df_playlist2['features'])

In [ ]:
df_playlist1

In [ ]:
df_playlist2

#### Drop or remove missing values

In [ ]:
playlist1features = pd.DataFrame.from_records(df_playlist1.features.dropna().tolist())
playlist2features = pd.DataFrame.from_records(df_playlist2.features.dropna().tolist())

#### Now is time to concat dataframes and drop features column

In [ ]:
df_1 = pd.concat([playlist1features, playlist2features], axis=0)

### Export dataframe to csv.

In [ ]:
df_1.to_csv('song_recommender_playlist_clusters.csv',index=False)

________________